### 프로젝트 실습

## A. 데이터 전처리

### A-1. 타입 통합 & 불량 데이터 처리

#### A-1-1. {데이터타입 통합}

kopo_channel_seasonality_new.csv 파일을 불러온 후 selloutData 변수에 담으세요.
이후 QTY 컬럼 → 실수 (float), 이외컬럼 → 문자(str)로 변경하세요

In [1]:
### 라이브러리 선언
import pandas as pd

In [2]:
### 파일 불러오기
selloutData = \
pd.read_csv("../dataset/kopo_channel_seasonality_new.csv")

In [3]:
selloutData

,REGIONID,PRODUCT,YEARWEEK,QTY
0,A60,PRODUCT4,201402,71.0
1,A60,PRODUCT59,201402,22275.0
2,A60,PRODUCT34,201402,4463.0
3,A60,PRODUCT47,201402,0.0
4,A60,PRODUCT56,201402,23.0
...,...,...,...,...
124653,A10,PRODUCT60,201630,824.0
124654,A10,PRODUCT56,201630,275.0
124655,A10,PRODUCT61,201630,0.0
124656,A10,PRODUCT12,201630,15021.0


In [4]:
### 데이터 타입 확인
selloutData.dtypes

REGIONID     object
PRODUCT      object
YEARWEEK      int64
QTY         float64
dtype: object

In [5]:
### 데이터 타입 변경
selloutData = selloutData.astype({"REGIONID":str, "PRODUCT":str, "YEARWEEK":str, "QTY":float})

In [6]:
### 데이터 타입 변경 검증
selloutData.dtypes

REGIONID     object
PRODUCT      object
YEARWEEK     object
QTY         float64
dtype: object

####  A-1-2. {불량 데이터 처리}

kopo_channel_seasonality_new.csv
자료를 담은
selloutData
변수에서
QTY
컬럼 음수(반품)인 경우 0, 양수인 경우 기존 QTY 값
유지하는 로직을 적용하여 QTY_NEW 컬럼을 추가하세요

In [7]:
### 불량 데이터 처리 방법 3가지
## 1. np.where 조건 사용 
import numpy as np

selloutData["QTY_NEW"] = \
np.where(selloutData.QTY < 0,
         0,
         selloutData.QTY)

In [8]:
## 2. loc 사용
selloutData.loc[selloutData.QTY < 0, "QTY_NEW"] = 0
selloutData.loc[selloutData.QTY >= 0, "QTY_NEW"] = selloutData.QTY

In [9]:
## 3. 함수 사용
def CleanQTY(inQTY) : 
    # inQTY = 3.4
    outQTY = 0

    if inQTY < 0 :
        outQTY = 0
    else :
        outQTY = inQTY
    return outQTY

In [10]:
selloutData["QTY_NEW"] = \
    selloutData.QTY.apply(CleanQTY)

In [11]:
### 데이터 검증
selloutData.loc[(selloutData.QTY < 0) & (selloutData.QTY_NEW != 0)].count()

REGIONID    0
PRODUCT     0
YEARWEEK    0
QTY         0
QTY_NEW     0
dtype: int64

### A-2. 데이터 통합

selloutData
자료에서
YEAR, WEEK
컬럼을 생성하고 WEEK 가 52 이하인
데이터만
조회한 후 refinedSelloutData 변수에 담으세요

In [12]:
### YEAR, WEEK 컬럼 생성하기
selloutData["YEAR"] = selloutData.YEARWEEK.str[:4]
selloutData["WEEK"] = selloutData.YEARWEEK.str[4:]

In [13]:
### WEEK 컬럼을 str로 비교...? (가능한건지 모르겠음... 값은 같음)
refinedSelloutData = selloutData.loc[selloutData.WEEK <= "52"]

In [14]:
### WEEK 컬럼 int로 변환하여 조건 걸기
refinedSelloutData = selloutData.loc[selloutData.WEEK.astype(int) <= 52]

In [15]:
refinedSelloutData

,REGIONID,PRODUCT,YEARWEEK,QTY,QTY_NEW,YEAR,WEEK
0,A60,PRODUCT4,201402,71.0,71.0,2014,02
1,A60,PRODUCT59,201402,22275.0,22275.0,2014,02
2,A60,PRODUCT34,201402,4463.0,4463.0,2014,02
3,A60,PRODUCT47,201402,0.0,0.0,2014,02
4,A60,PRODUCT56,201402,23.0,23.0,2014,02
...,...,...,...,...,...,...,...
124653,A10,PRODUCT60,201630,824.0,824.0,2016,30
124654,A10,PRODUCT56,201630,275.0,275.0,2016,30
124655,A10,PRODUCT61,201630,0.0,0.0,2016,30
124656,A10,PRODUCT12,201630,15021.0,15021.0,2016,30


In [16]:
### 데이터 검증
refinedSelloutData.loc[selloutData.WEEK.astype(int) > 52].count()

REGIONID    0
PRODUCT     0
YEARWEEK    0
QTY         0
QTY_NEW     0
YEAR        0
WEEK        0
dtype: int64

## B. 추세선 도출

### B-1 대표값 생성

refinedSelloutData
에서
지역
, 상품 , 연주차 컬럼순으로 오름차순 정렬하여
sortedData
변수에 담으세요

In [17]:
### 정렬 기본 키 설정
sortkeys = ["REGIONID","PRODUCT","YEARWEEK"]

In [18]:
### 기본키를 기준으로 오름차순 정렬
sortedData = refinedSelloutData.sort_values(by = sortkeys,
                              ascending = True,
                              ignore_index = True)

In [19]:
sortedData

,REGIONID,PRODUCT,YEARWEEK,QTY,QTY_NEW,YEAR,WEEK
0,A00,PRODUCT34,201401,661.0,661.0,2014,01
1,A00,PRODUCT34,201402,679.0,679.0,2014,02
2,A00,PRODUCT34,201403,578.0,578.0,2014,03
3,A00,PRODUCT34,201404,532.0,532.0,2014,04
4,A00,PRODUCT34,201405,516.0,516.0,2014,05
...,...,...,...,...,...,...,...
123859,A77,PRODUCT12,201648,4152.0,4152.0,2016,48
123860,A77,PRODUCT12,201649,5086.0,5086.0,2016,49
123861,A77,PRODUCT12,201650,5846.0,5846.0,2016,50
123862,A77,PRODUCT12,201651,4933.0,4933.0,2016,51


[
지역 , 상품 , 연도 별 집계
sortedData
에서 지역 , 상품, 연도 단위
판매량
(QTY_NEW) 의 평균 연산 후
groupData
변수에 담으세요
이후
컬럼명을 QTY_MEAN 로 변경하세요

In [20]:
### 그룹으로 묶을 기본 키 설정
groupKey = ["REGIONID","PRODUCT","YEAR"]

In [21]:
### 기본키로 그룹을 묶고 QTY_NEW의 평균 연산
groupData = sortedData.groupby(by = groupKey)\
                    ["QTY_NEW"].\
                    agg(["mean"])

In [22]:
groupData

mean
REGIONID PRODUCT   YEAR             
A00      PRODUCT34 2014   275.961538
                   2015    86.634615
                   2016    36.576923
         PRODUCT58 2014     2.673077
                   2015     5.711538
...                              ...
A77      PRODUCT1  2015  3030.019231
                   2016  3375.326923
         PRODUCT12 2014  2035.788462
                   2015  3540.980769
                   2016  4837.153846

[2382 rows x 1 columns]

In [23]:
### 그룹화한 데이터 인덱스 조정 및 컬럼이름 변경
groupDataFianl=\
groupData.reset_index().\
rename(columns = {"mean":"QTY_MEAN"})

In [24]:
groupData = groupDataFianl

In [25]:
groupData

,REGIONID,PRODUCT,YEAR,QTY_MEAN
0,A00,PRODUCT34,2014,275.961538
1,A00,PRODUCT34,2015,86.634615
2,A00,PRODUCT34,2016,36.576923
3,A00,PRODUCT58,2014,2.673077
4,A00,PRODUCT58,2015,5.711538
...,...,...,...,...
2377,A77,PRODUCT1,2015,3030.019231
2378,A77,PRODUCT1,2016,3375.326923
2379,A77,PRODUCT12,2014,2035.788462
2380,A77,PRODUCT12,2015,3540.980769


refinedSelloutData와 groupData 를
[REGIONID, PRODUCT, YEAR] 키로 조인하여
mergedData 변수에 담으세요

In [26]:
### 키값 설정 및 조인
joinKey = ["REGIONID", "PRODUCT", "YEAR"]

mergedData =pd.merge(left = refinedSelloutData,
                     right = groupData,
                     on = joinKey,
                     how = "inner")

In [27]:
### QTY값을 제외한 나머지 키값을 기준으로 정렬
sotKeys = ["REGIONID","PRODUCT","YEARWEEK","YEAR","WEEK"]
mergedData = mergedData.sort_values(by = sotKeys, 
                                    ascending = [True,True,True,True,True])

In [28]:
mergedData

,REGIONID,PRODUCT,YEARWEEK,QTY,QTY_NEW,YEAR,WEEK,QTY_MEAN
10661,A00,PRODUCT34,201401,661.0,661.0,2014,01,275.961538
10662,A00,PRODUCT34,201402,679.0,679.0,2014,02,275.961538
10663,A00,PRODUCT34,201403,578.0,578.0,2014,03,275.961538
10660,A00,PRODUCT34,201404,532.0,532.0,2014,04,275.961538
10665,A00,PRODUCT34,201405,516.0,516.0,2014,05,275.961538
...,...,...,...,...,...,...,...,...
96903,A77,PRODUCT12,201648,4152.0,4152.0,2016,48,4837.153846
96893,A77,PRODUCT12,201649,5086.0,5086.0,2016,49,4837.153846
96904,A77,PRODUCT12,201650,5846.0,5846.0,2016,50,4837.153846
96898,A77,PRODUCT12,201651,4933.0,4933.0,2016,51,4837.153846


## C 계절성 지수 산출

### C-1 계절성 지수 계산

계절성 지수는 QTY_NEW / QTY_MEAN 으로 산출하여 SEASONALITY 컬럼을 생성한다

In [29]:
### 계절성 지수 산출
mergedData.loc[:,"SEASONALITY"] = (mergedData.QTY_NEW) / (mergedData.QTY_MEAN)

In [30]:
mergedData

,REGIONID,PRODUCT,YEARWEEK,QTY,QTY_NEW,YEAR,WEEK,QTY_MEAN,SEASONALITY
10661,A00,PRODUCT34,201401,661.0,661.0,2014,01,275.961538,2.395261
10662,A00,PRODUCT34,201402,679.0,679.0,2014,02,275.961538,2.460488
10663,A00,PRODUCT34,201403,578.0,578.0,2014,03,275.961538,2.094495
10660,A00,PRODUCT34,201404,532.0,532.0,2014,04,275.961538,1.927805
10665,A00,PRODUCT34,201405,516.0,516.0,2014,05,275.961538,1.869826
...,...,...,...,...,...,...,...,...,...
96903,A77,PRODUCT12,201648,4152.0,4152.0,2016,48,4837.153846,0.858356
96893,A77,PRODUCT12,201649,5086.0,5086.0,2016,49,4837.153846,1.051445
96904,A77,PRODUCT12,201650,5846.0,5846.0,2016,50,4837.153846,1.208562
96898,A77,PRODUCT12,201651,4933.0,4933.0,2016,51,4837.153846,1.019815


### C-2 주차별 효과 계산

마지막으로 [지역 상품 , 주차] 별 계절성 지수 평균 값을 조별 산출하여 finalResult 변수에 담으세요

In [31]:
### 그룹키 설정 및 평균값 산출, rename
groupKeys = ["REGIONID","PRODUCT","WEEK"]

finalResult = mergedData.groupby(by = groupKeys)["SEASONALITY"].agg(["mean"])\
                        .reset_index().rename(columns = {"mean":"SEASONALITY"})

In [32]:
finalResult

,REGIONID,PRODUCT,WEEK,SEASONALITY
0,A00,PRODUCT34,01,1.570782
1,A00,PRODUCT34,02,1.755540
2,A00,PRODUCT34,03,1.319460
3,A00,PRODUCT34,04,1.490298
4,A00,PRODUCT34,05,1.061909
...,...,...,...,...
41283,A77,PRODUCT12,48,1.352712
41284,A77,PRODUCT12,49,1.094083
41285,A77,PRODUCT12,50,1.386116
41286,A77,PRODUCT12,51,1.255192
